In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import model_selection
from sklearn.metrics import f1_score,roc_auc_score

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
import lightgbm
from sklearn.decomposition import PCA

from contextlib import contextmanager
import time
import gc

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def read_df():
    df_train = pd.read_csv('/content/drive/My Drive/financal_inclusion/Train_v2.csv')
    df_test = pd.read_csv('/content/drive/My Drive/financal_inclusion/Test_v2.csv')
    df_train.loc[df_train['bank_account'] == 'Yes', 'bank_account'] = 1
    df_train.loc[df_train['bank_account'] == 'No', 'bank_account'] = 0
    return df_train, df_test

df_train, df_test = read_df()

In [ ]:
df_train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,1,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,0,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,1,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,0,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,0,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [ ]:
def feature_engineering(df):
    df['f1'] = df.groupby(['age_of_respondent'])['age_of_respondent'].transform('count')
    df['f2'] = df.groupby(['age_of_respondent'])['age_of_respondent'].transform('std') 
    df['f3'] = df.groupby(['household_size'])['household_size'].transform('count')
    #df['f4'] = df.groupby(columns)['age_of_respondent'].transform('count')
    df['f5'] = df['age_of_respondent'] / df.groupby('household_size')['age_of_respondent'].transform('mean')
    df['f4'] = df.groupby(['age_of_respondent', 'education_level'])['education_level'].transform('count')
    df['f4'] = df.groupby(['household_size'])['household_size'].transform('std')
    df['f5'] = df.groupby(['age_of_respondent'])['education_level'].transform('count')
    
    df['f5'] = df.groupby(['job_type'])['job_type'].transform('count') 
    df['f6'] = df.groupby(['job_type'])['age_of_respondent'].transform('mean') 
    df['f7'] = df.groupby(['job_type'])['age_of_respondent'].transform('std')
    df['f5'] = df.groupby(['household_size'])['household_size'].transform('count') 
    #df['f6'] = (df['household_size'] / df.groupby(['age_of_respondent'])['household_size'].transform('mean') 
    #    df.groupby(['age_of_respondent'])['household_size'].transform('std')     
    return df

df_train, df_test = read_df()
df_train = feature_engineering(df_train)    
df_test = feature_engineering(df_test)    

In [ ]:
df_train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,f1,f2,f3,f5,f4,f6,f7
0,Kenya,2018,uniqueid_1,1,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,562,0.0,3920,3920,0.0,38.807830,14.609445
1,Kenya,2018,uniqueid_2,0,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent,275,0.0,2853,2853,0.0,66.291498,18.699051
2,Kenya,2018,uniqueid_3,1,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,615,0.0,2853,2853,0.0,38.807830,14.609445
3,Kenya,2018,uniqueid_4,0,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,522,0.0,2853,2853,0.0,34.079621,11.558256
4,Kenya,2018,uniqueid_5,0,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed,615,0.0,751,751,0.0,36.160622,13.458767


In [ ]:
def preprocessing(df):
    print("Starting Preprocessing..")
    # Threshold for removing correlated variables
    threshold = 0.97
    
    # Absolute value correlation matrix
    corr_matrix = df.corr().abs()
    
    # Getting the upper triangle of correlations
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    
    # Select columns with correlations above threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    print('There are %d columns to remove.' % (len(to_drop)))
    print('Columns to drop: ', to_drop)
    
    df = df.drop(columns = to_drop)
    print('Shape after drop high correlated features: ', df.shape)
    print('Remaining columns: ', df.columns)
    
    return df.columns.tolist()

df_train, df_test = read_df()
df_train = feature_engineering(df_train)    
df_test = feature_engineering(df_test)
columns = ['country', 'year', 'location_type','cellphone_access',
       'household_size', 'age_of_respondent', 'gender_of_respondent',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5','f4','f7','f3']
columns = preprocessing(df_train[columns])
df_train = df_train[columns + ['bank_account']]
df_test = df_test[columns]

Starting Preprocessing..
There are 1 columns to remove.
Columns to drop:  ['f3']
Shape after drop high correlated features:  (23524, 15)
Remaining columns:  Index(['country', 'year', 'location_type', 'cellphone_access',
       'household_size', 'age_of_respondent', 'gender_of_respondent',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5', 'f4', 'f7'],
      dtype='object')


In [ ]:
df_train.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,f1,f5,f4,f7,bank_account
0,Kenya,2018,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,562,3920,0.0,14.609445,1
1,Kenya,2018,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent,275,2853,0.0,18.699051,0
2,Kenya,2018,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,615,2853,0.0,14.609445,1
3,Kenya,2018,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,522,2853,0.0,11.558256,0
4,Kenya,2018,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed,615,751,0.0,13.458767,0


In [ ]:
df_train.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,f1,f5,f4,f7,bank_account
0,Kenya,2018,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,562,3920,0.0,14.609445,1
1,Kenya,2018,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent,275,2853,0.0,18.699051,0
2,Kenya,2018,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,615,2853,0.0,14.609445,1
3,Kenya,2018,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,522,2853,0.0,11.558256,0
4,Kenya,2018,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed,615,751,0.0,13.458767,0


In [ ]:
def encoding(df, list_cols):
    for col in list_cols:
        lbl = LabelEncoder()
        lbl.fit(list(df[col].values))
        df[col] = lbl.transform(list(df[col].values))      
        df_dummies = pd.get_dummies(df[col], drop_first=True, prefix=(str(col)))
        df = pd.concat([df, df_dummies], axis=1)
        
    return df

df_train, df_test = read_df()
df_train = feature_engineering(df_train)    
df_test = feature_engineering(df_test)
columns = ['country', 'year', 'location_type',
       'household_size', 'age_of_respondent', 'gender_of_respondent','cellphone_access',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5','f4','f7','f3']
columns = preprocessing(df_train[columns])
df_train = df_train[columns + ['bank_account']]
df_test = df_test[columns + ['uniqueid']]
categorical_columns = ['country', 'location_type','cellphone_access', 
                'gender_of_respondent', 'relationship_with_head', 
                'marital_status', 'education_level', 'job_type',
             ]
df_train = encoding(df_train, categorical_columns)
df_test = encoding(df_test, categorical_columns)

Starting Preprocessing..
There are 1 columns to remove.
Columns to drop:  ['f3']
Shape after drop high correlated features:  (23524, 15)
Remaining columns:  Index(['country', 'year', 'location_type', 'household_size',
       'age_of_respondent', 'gender_of_respondent', 'cellphone_access',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5', 'f4', 'f7'],
      dtype='object')


In [ ]:
df_train.head()

,country,year,location_type,household_size,age_of_respondent,gender_of_respondent,cellphone_access,relationship_with_head,marital_status,education_level,job_type,f1,f5,f4,f7,bank_account,country_1,country_2,country_3,location_type_1,cellphone_access_1,gender_of_respondent_1,relationship_with_head_1,relationship_with_head_2,relationship_with_head_3,relationship_with_head_4,relationship_with_head_5,marital_status_1,marital_status_2,marital_status_3,marital_status_4,education_level_1,education_level_2,education_level_3,education_level_4,education_level_5,job_type_1,job_type_2,job_type_3,job_type_4,job_type_5,job_type_6,job_type_7,job_type_8,job_type_9
0,0,2018,0,3,24,0,1,5,2,3,9,562,3920,0.0,14.609445,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,2018,0,5,70,0,0,1,4,0,4,275,2853,0.0,18.699051,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,2018,1,5,26,1,1,3,3,5,9,615,2853,0.0,14.609445,1,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,0,2018,0,5,34,0,1,1,2,2,3,522,2853,0.0,11.558256,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
4,0,2018,1,8,26,1,0,0,3,2,5,615,751,0.0,13.458767,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
import lightgbm


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators':[1500,2000]}
clf = LGBMClassifier(
            n_estimators=2000,
            learning_rate=0.1,
            num_leaves=123,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=15,
            reg_alpha=.1,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2
        )
columns = ['country', 'year', 'location_type',
       'household_size', 'age_of_respondent', 'gender_of_respondent','cellphone_access',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5','f4','f7','f3']
categorical_columns = ['country', 'location_type','cellphone_access', 
                'gender_of_respondent', 'relationship_with_head', 
                'marital_status', 'education_level', 'job_type',
             ]
clfGrid = GridSearchCV(LGBMClassifier, param_grid)
GridSearchCV(clf, param_grid, scoring=None)
x=df_train[columns]
x=encoding(x,categorical_columns)
clfGrid.fit(x,df_train['bank_account'])

TypeError: ignored

In [ ]:
def cross_validation(df): 
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    train_x = df.drop('bank_account', axis=1)
    train_y = df['bank_account']
    score = []
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
        
        trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
        val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
        
        clf = LGBMClassifier(
            n_estimators=2000,
            learning_rate=0.1,
            num_leaves=123,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=15,
            reg_alpha=.1,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2
        )

        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                eval_metric='auc', verbose=250, early_stopping_rounds=150,
               )
        score.append(clf.best_score_['valid_1']['auc'])
        #lightgbm.plot_importance(clf, height=1.0, max_num_features=10)    

        del clf, trn_x, trn_y, val_x, val_y   
    
    return score

df_train, df_test = read_df()
df_train = feature_engineering(df_train)    
df_test = feature_engineering(df_test)
columns = ['country', 'year', 'location_type',
       'household_size', 'age_of_respondent', 'gender_of_respondent','cellphone_access',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5','f4','f7','f3']
columns = preprocessing(df_train[columns])
df_train = df_train[columns + ['bank_account']]
df_test = df_test[columns + ['uniqueid']]
df_test['countryid'] = df_test['country']
categorical_columns = ['country', 'location_type','cellphone_access', 
                'gender_of_respondent', 'relationship_with_head', 
                'marital_status', 'education_level', 'job_type',
             ]
df_train = encoding(df_train, categorical_columns)
df_test = encoding(df_test, categorical_columns)
score = cross_validation(df_train)
np.mean(score), np.std(score)

Starting Preprocessing..
There are 1 columns to remove.
Columns to drop:  ['f3']
Shape after drop high correlated features:  (23524, 15)
Remaining columns:  Index(['country', 'year', 'location_type', 'household_size',
       'age_of_respondent', 'gender_of_respondent', 'cellphone_access',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5', 'f4', 'f7'],
      dtype='object')
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[20]	training's auc: 0.912459	training's binary_logloss: 0.249916	valid_1's auc: 0.859931	valid_1's binary_logloss: 0.287843
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[23]	training's auc: 0.914733	training's binary_logloss: 0.246106	valid_1's auc: 0.864504	valid_1's binary_logloss: 0.283124
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[31]	training's auc: 0.925732	

(0.8584567504488874, 0.006957305118417169)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators':[1000,2000]}
clf = LGBMClassifier(
            n_estimators=2000,
            learning_rate=0.1,
            num_leaves=123,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=15,
            reg_alpha=.1,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2
        )
columns = ['country', 'year', 'location_type',
       'household_size', 'age_of_respondent', 'gender_of_respondent','cellphone_access',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5','f4','f7','f3']
categorical_columns = ['country', 'location_type','cellphone_access', 
                'gender_of_respondent', 'relationship_with_head', 
                'marital_status', 'education_level', 'job_type',
             ]
clfGrid = GridSearchCV(clf, param_grid)
GridSearchCV(clf, param_grid, scoring=None)
x=df_train[columns]
x=encoding(x,categorical_columns)
clfGrid.fit(x,df_train['bank_account'],eval_metric='auc')

KeyError: ignored

In [ ]:
print(clfGrid.best_score_,clfGrid.best_params_,clfGrid.best_estimator_)

0.7196055092671314 {'n_estimators': 1000} LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=2, min_split_gain=0.01,
               n_estimators=1000, n_jobs=-1, num_leaves=123, objective=None,
               random_state=None, reg_alpha=0.1, reg_lambda=0.1, silent=True,
               subsample=0.9, subsample_for_bin=200000, subsample_freq=0)


In [ ]:
score

[0.8599305206585764,
 0.8645044653832537,
 0.845823722514803,
 0.8649250841989922,
 0.8570999594888116]

In [ ]:
def train(df):
    train_x = df.drop('bank_account', axis=1)
    train_y = df['bank_account']     
    train_x, val_x, train_y, val_y =\
     train_test_split(train_x, train_y, stratify=train_y, test_size=0.2, random_state=123)   
    clf = LGBMClassifier(
        n_estimators=2000,
        learning_rate=0.1,
        num_leaves=123,
        colsample_bytree=.8,
        subsample=.9,
        max_depth=15,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2
    )

    clf.fit(train_x, train_y, 
            eval_set= [(train_x, train_y), (val_x, val_y)], 
            eval_metric='auc', verbose=250, early_stopping_rounds=150,
            )
    return clf

In [ ]:
def predict(clf, df):
    df['uniqueid'] = df['uniqueid'].apply(str) + ' x ' + df['countryid'].apply(str)
    df['bank_account'] = clf.predict(df.drop(columns=['uniqueid', 'countryid']))
    df[['uniqueid', 'bank_account']].to_csv('results.csv', index= False)
    return df[['uniqueid', 'bank_account']]

In [ ]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0)) 
  
with timer("Importing Datasets: "):    
    df_train, df_test = read_df()
    gc.collect();

with timer("Feature Engineering: "):
    df_train = feature_engineering(df_train)
    df_test = feature_engineering(df_test)
    gc.collect();

with timer("Feature Preprocessing: "):
    columns = ['country', 'year', 'location_type', 'cellphone_access',
          'household_size', 'age_of_respondent', 'gender_of_respondent',
          'relationship_with_head', 'marital_status', 'education_level',
          'job_type', 'f1', 'f5']  
    columns = preprocessing(df_train[columns])
    df_train = df_train[columns + ['bank_account']]
    df_test = df_test[columns + ['uniqueid']]
    df_test['countryid'] = df_test['country']
    categorical_columns = ['country', 'location_type', 'cellphone_access', 
                    'gender_of_respondent', 'relationship_with_head', 
                    'marital_status', 'education_level', 'job_type',
                ]
    df_train = encoding(df_train, categorical_columns)
    df_test = encoding(df_test, categorical_columns)    
    gc.collect()

with timer("Run LightGBM with cross validation"):
    score = cross_validation(df_train)
    print('Score mean: %.3f, std: %.3f' % (np.mean(score), np.std(score)))
    gc.collect()

with timer("Training"):
    clf = train(df_train)
    gc.collect()

with timer("Prediction"):
    preds = predict(clf, df_test)
    gc.collect()

Importing Datasets:  - done in 0s
Feature Engineering:  - done in 0s
Starting Preprocessing..
There are 0 columns to remove.
Columns to drop:  []
Shape after drop high correlated features:  (23524, 13)
Remaining columns:  Index(['country', 'year', 'location_type', 'cellphone_access',
       'household_size', 'age_of_respondent', 'gender_of_respondent',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'f1', 'f5'],
      dtype='object')
Feature Preprocessing:  - done in 0s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[16]	training's auc: 0.907403	training's binary_logloss: 0.259994	valid_1's auc: 0.864818	valid_1's binary_logloss: 0.289053
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[29]	training's auc: 0.920979	training's binary_logloss: 0.237226	valid_1's auc: 0.862375	valid_1's binary_logloss: 0.281587
Training until validation scores don't